# Analysis for an Anemometer
Developed for the ATMOS22, which small size made it ideal for climatecell experiments.

In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Load datasets, and check it out:

In [2]:
columns = ['datetime','windSpeed','windDirection','gustWindSpeed','airTemperature','xOrientation','yOrientation','NorthWindSpeed','EastWindSpeed']
file = "test_data/Tests_G4_G7/LOGGER32.csv"
wind_df = pd.read_csv(file, names=columns,header=None, skiprows=1)
#wind_df = pd.concat(map(lambda file, i: pd.read_csv(file,names=columns,header=None, skiprows=1), input_files, index))
wind_df['datetime'] =  pd.to_datetime(wind_df['datetime'])

In [ ]:
wind_df.dtypes

In [ ]:
wind_df

It looks like the tilt sensor has a timing issue, occasionally retuning: -9990.0. Ardiuno retransmits if this happens, but still good to check for it.  

In [ ]:
# Drop the tow with a -9990 value
rows_with_negative_990 = wind_df[(wind_df == -9990.0).any(axis=1)]
wind_df = wind_df.drop(rows_with_negative_990.index)
rows_with_negative_990

In [ ]:
# Tilt
fig = make_subplots()
fig.add_trace(go.Scatter(x = wind_df['datetime'], y=wind_df['xOrientation'], mode = 'lines', name = 'xOrientation'))
fig.add_trace(go.Scatter(x = wind_df['datetime'], y=wind_df['yOrientation'], mode = 'lines', name = 'yOrientation'))
fig.update_layout(title_text=f"Anemometer tilt")
fig.update_layout(xaxis_title="Datetime", yaxis_title="Tilt (degrees)")

# Interactivity
fig.update_layout(
        hovermode="x unified", #show trace line
        xaxis=dict( # Buttons at the top
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="day", step="day",stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider = dict(visible=True), type="date")
)
# Save graph
fig.write_html('tilt.html')
# show graph
fig.show()

In [ ]:
# Windspeed and direction
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x = wind_df['datetime'], y=wind_df['windSpeed'], mode = 'lines', name = 'windspeed'), secondary_y=False)
fig.add_trace(go.Scatter(x = wind_df['datetime'], y=wind_df['NorthWindSpeed'], mode = 'lines', name = 'NorthWindSpeed'), secondary_y=False)
fig.add_trace(go.Scatter(x = wind_df['datetime'], y=wind_df['EastWindSpeed'], mode = 'lines', name = 'EastWindSpeed'), secondary_y=False)
fig.add_trace(go.Scatter(x = wind_df['datetime'], y=wind_df['gustWindSpeed'], mode = 'lines', name = 'gusts'), secondary_y=False)
fig.add_trace(go.Scatter(x = wind_df['datetime'], y=wind_df['windDirection'], mode = 'lines', name = 'windDirection', opacity=0.5), secondary_y=True)
fig.update_layout(title_text=f"Windspeed and direction")
fig.update_layout(xaxis_title="Datetime", yaxis_title="Windspeed (m/s)")
fig.update_yaxes(title_text="winddirection (degrees)", secondary_y=True)

# Interactivity
fig.update_layout(
        hovermode="x unified", #show trace line
        xaxis=dict( # Buttons at the top
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="day", step="day",stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider = dict(visible=True), type="date"),
        yaxis = dict(
             fixedrange = False
)
)
# Save graph
fig.write_html('windspeed.html')
# show graph
fig.show()

In [ ]:
# Airtemperature & Windspeed
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x = wind_df['datetime'], y=wind_df['windSpeed'], mode = 'lines', name = 'windspeed'), secondary_y=False)
fig.add_trace(go.Scatter(x = wind_df['datetime'], y=wind_df['NorthWindSpeed'], mode = 'lines', name = 'NorthWindSpeed'), secondary_y=False)
fig.add_trace(go.Scatter(x = wind_df['datetime'], y=wind_df['EastWindSpeed'], mode = 'lines', name = 'EastWindSpeed'), secondary_y=False)
fig.add_trace(go.Scatter(x = wind_df['datetime'], y=wind_df['gustWindSpeed'], mode = 'lines', name = 'gusts'), secondary_y=False)
fig.add_trace(go.Scatter(x = wind_df['datetime'], y=wind_df['airTemperature'], mode = 'lines', name = 'Temperature'), secondary_y=True)
fig.update_layout(title_text=f"Air temperature")
fig.update_layout(xaxis_title="Datetime", yaxis_title="Temperature (degrees)")

# Interactivity
fig.update_layout(
        hovermode="x unified", #show trace line
        xaxis=dict( # Buttons at the top
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="day", step="day",stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider = dict(visible=True), type="date"),
        yaxis = dict(
             fixedrange = False
)
)
# Save graph
fig.write_html('airtemperature.html')
# show graph
fig.show()

In [ ]:
# wind rose chart (also known as a polar bar chart)
# ref: https://plotly.com/python/wind-rose-charts/
# can only show a few data points, therefore it is advised to either select 10 points or less
# Alternatively the data can be resampled
wind_subdf = wind_df.head(10)

#wind_subdf = wind_df.head(1000)
#wind_subdf.set_index('datetime', inplace=True)
#wind_subdf = wind_subdf.resample('H').mean()

fig = go.Figure()
fig.add_trace(go.Barpolar(
    r=wind_subdf['windSpeed'],
    theta=wind_subdf['windDirection'],
    name='Wind Speed',
    marker_color='blue',
    marker_line_width=5
))

# Add gustWindSpeed as an overlay
fig.add_trace(go.Barpolar(
    r=wind_subdf['gustWindSpeed'],
    theta=wind_subdf['windDirection'],
    name='Gust Wind Speed',
    marker_color='red',
    marker_line_width=5,
    opacity=0.5
    
))

fig.update_layout(
    title='Wind Rose Chart with Gust Wind Speed Overlay',
    polar=dict(
        radialaxis=dict(visible=True)
    )
)
# Save graph
fig.write_html('wind_rose.html')
# show graph
fig.show()